# Chapter 2

## Fig 2.04 Exponential decay

In [ ]:
using Plots
Plots.default(linewidth=2)

In [ ]:
plot([t-> 3 * exp(-t) t->3 * exp(-2t) t-> 3 * exp(-3t)], 0.0, 5.0, 
     xlim = (0, 5), ylim=(0, 3.2),
     xlabel="Time", ylabel="Concentration", 
     label = ["exp(-t)" "exp(-2t)" "exp(-3t)"], 
     title= "Figure 2.4"
)

## Fig 2.09 

Numerical Simulation of a metabolic network using Caatalyst.jl

In [ ]:
using OrdinaryDiffEq
using Catalyst
using Plots
Plots.default(linewidth=2)

In [ ]:
# Convenience functions
hill(x, k) = x / (x + k)
hill(x, k, n) = hill(x^n, k^n)

In [ ]:
# Model building
net = @reaction_network begin
    3.0, 0 --> A
    2.0, A --> B
    2.5, A + B --> C + D
    3.0, C --> 0
    3.0, D --> 0
end

In [ ]:
u0 = zeros(4)
tend = 10.0
sol = solve(ODEProblem(net, u0, tend), Tsit5())

In [ ]:
plot(sol, xlims=(0.0, 4.0), ylims=(0.0, 1.0), 
     xlabel="Time (sec)", ylabel="Concentration (mM)", title="Figure 2.09",
     legend=:bottomright)

## Figure 2.11-14 

Model reduction of ODE metabolic networks

In [ ]:
using OrdinaryDiffEq
using ComponentArrays
using UnPack
using Plots
Plots.default(linewidth=2)

In [ ]:
function full_model!(D, u, p, t)
    @unpack k0, k1, km1, k2 = p
    @unpack a, b = u
    vab = k1 * a - km1 * b
    D.a = k0 - vab
    D.b = vab - k2 * b
end

In [ ]:
ps1 = ComponentArray(k0=0., k1=9., km1 = 12., k2 = 2.)
u0 = ComponentArray(a=0., b=10.)
tend = 3.0

In [ ]:
sol1_full = solve(ODEProblem(full_model!, u0, tend, ps1), Tsit5())

In [ ]:
plot(sol1_full, xlabel="Time (AU)", ylabel="Concentration (AU)", title="Fig. 2.11 (Full model)")

## Figure 2.12 : Rapid equilibrium assumption

In [ ]:
_a(u, p) = hill(p.km1, p.k1) * u
_b(u, p) = hill(p.k1, p.km1) * u
function re_model(u, p, t)
    @unpack k0, k1, km1, k2 = p
    b = _b(u, p)
    return k0 - k2 * b
end

In [ ]:
sol1_re = solve(ODEProblem(re_model, sum(u0), tend, ps1), Tsit5())

In [ ]:
pl2 = plot(sol1_full, line=(:dash, 1),label=["A (full solution)" "B (full solution)"])

plot!(pl2, t -> _a(sol1_re(t), ps1), 0, tend, label="A (rapid equilibrium)")
plot!(pl2, t -> _b(sol1_re(t), ps1), 0, tend, label="B (rapid equilibrium)")
plot!(pl2, title="Fig. 2.12 (Rapid equilibrium model)",
        xlabel="Time (AU)", ylabel="Concentration (AU)")

## Figure 2.13: Rapid equilibrium 

with another set of parameters not suitable for Rapid equilibrium assumption.

In [ ]:
ps2 = ComponentArray(k0=9., k1=20., km1 = 12., k2 = 2.)
u0 = ComponentArray(a=8., b=4.)
tend = 3.0
sol2full = solve(ODEProblem(full_model!, u0, tend, ps2), Tsit5())
sol2re = solve(ODEProblem(re_model, sum(u0), tend, ps2), Tsit5())

In [ ]:
pl3 = plot(sol2full, line=(:dash, 1),label=["A (full solution)" "B (full solution)"])
plot!(pl3, [t -> _a(sol2re(t), ps2) t -> _b(sol2re(t), ps2)], 0, tend, label=["A (rapid equilibrium)" "B (rapid equilibrium)"])
plot!(pl3, title="Fig. 2.13 (Rapid equilibrium model)", xlabel="Time (AU)", ylabel="Concentration (AU)")

## Figure 2.14 : QSSA

Quasi-steady state assumption on species A

In [ ]:
function qss_model(b, p, t)
    @unpack k0, k2 = p
    return k0 - b * k2
end

# The initial conditions in QSSA is not trivial
function qss_u0(u0, p)
    @unpack k0, k1, km1, k2 = p
    return (k1 * sum(u0) - k0) / (k1 + km1)
end

function qss_a(b, p)
    @unpack k0, k1, km1, k2 = p
    return (k0 + km1 * b)/k1
end

In [ ]:
u0qss = qss_u0(u0, ps2)
sol2qss = solve(ODEProblem(qss_model, u0qss, tend, ps2), Tsit5())

In [ ]:
pl4 = plot(sol2full, line=(:dash), xlims=(0.0, tend),
    xlabel="Time (arbitrary units)",
    ylabel="Concentration (arbitrary units)",
    title="Figure 2.14: Ref vs QSSA")

plot!(pl4, sol2qss, label="B (QSSA)", line=(:red))
plot!(pl4, t -> qss_a(sol2qss(t), ps2), 0, tend, label="A (QSSA)", line=(:blue))

## Problem 2.4.6

In [ ]:
using Plots
using OrdinaryDiffEq
Plots.default(linewidth=2)

In [ ]:
# Model
f(u, p, t) = p * (1.0 - u)

In [ ]:
p = 1.0
u0 = 0.0
tspan = 10.0

prob = ODEProblem(f, u0, tspan, p)
sol = solve(prob, Tsit5())

## Runtime information

In [ ]:
versioninfo()

In [ ]:
using Pkg
Pkg.status()